# Bollinger Bands & RSI
* This is a strategy built based on VWAP by replacing the SMA (simple moving average) with VWAP for Bollinger Bands, which may filter noise better than SMA-based bands, especially for intraday strategies.

## 1 Notations

### 1.1 VWAP
* $VWAP_{t} = \frac{\sum_{k \in S}^{t} TP_{k} \cdot V_{k}}{\sum_{k \in S}^{t} V_{k}}$
* $Z_{t} = \frac{CP_{t} - VWAP_{t}}{\sigma_{t}}$
* $V_{t}$ = Volume at t-th minute bar
* $TP_{t}$ = Typical price = $\frac{HP_{t} + LP_{t} + CP_{t}}{3}$
* $HP_{t}$ = High price
* $LP_{t}$ = Low price
* $CP_{t}$ = Closing price at t-th minute bar
* $\sigma_{t}$ = Rolling standard deviation of ($CP_{t} - VWAP_{t}$)

### 1.2 Bollinger Bands
* $MA_{t} = \frac{1}{n_{BB}}\sum_{k \in S}^{t} P_{k}$
* $BB_{upper,t} = VWAP_{t} + k \cdot \sigma_{t}$
* $BB_{lower,t} = VWAP_{t} - k \cdot \sigma_{t}$
* $k$ = Standard deviation multiplier

### 1.3 RSI
* $\Delta P_{t} = P_{t} - P_{t-1}$
* $G_{t}$ = Separate gain = $max(\Delta P_{t}, 0)$
* $L_{t}$ = Separate loss = $max(-\Delta P_{t}, 0)$
* $AG_{t}$ = Smoothed average gain = $\frac{AG_{t-1} \cdot (n_{RSI} - 1) + G_{t}}{n_{RSI}}$
* $AL_{t}$ = Smoothed avergae loss = $\frac{AL_{t-1} \cdot (n_{RSI} - 1) + L_{t}}{n_{RSI}}$
* $RS_{t}$ = Relative strength = $\frac{AG_{t}}{AL_{t}}$
* $RSI_{t} = 100 - \frac{100}{1 + RS_{t}}$

## 2 Mean Reversion Strategy
* Position: long only (for now)
* Entry: if $P_{t} < BB_{lower,t}$ ($Z_{t} < -2$) and $RSI_{t} < 30$
* Exit: if $P_{t} \ge VWAP_{t}$ ($Z_{t} \ge 0$) or $RSI_{t} \ge 50$
* Look back period for Bollinger Bands $n_{BB} = 30$ (rolling standard deviation)
* Look back period for RSI $n_{RSI} = 13$
* Standard deviation multiplier $k = 2$

## 3 Codes

In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt

### 3.1 Data Management
* Each csv file has its name in the `"YYYY-MM-DD.csv"` format
* It has columns: `["ticker", "volume", "open", "close", "high", "low", "window_start", "transactions"]`
* `"ticker"` includes the minute bar aggregate of every ticker symbol of US stocks on the given day
* `"window_start"` is epoch time in ns

In [2]:
def datafeed(file_name, ticker_symbol):
    df = pd.read_csv(file_name)
    df = df[df["ticker"] == ticker_symbol]
    df["window_start"] = pd.to_datetime(df["window_start"], unit="ns")
    df = df.set_index("window_start").sort_index()
    df = df.between_time("09:30", "16:00")
    return df

### 3.2 VWAP
* For the rolling standard deviation, we choose `window=30` according $n_{BB} = 30$

In [3]:
def compute_typical_price(df):
    df["typical"] = (df["high"] + df["low"] + df["close"]) / 3
    return df

In [4]:
def compute_vwap(df):
    df["vwap"] = ta.vwap(high=df["high"], low=df["low"], close=df["close"], volume=df["volume"])
    return df

In [5]:
def compute_rolling_std(df, n_bb=30):
    df["rolling_std"] = (df["close"] - df["vwap"]).rolling(window=n_bb).std()
    return df

In [6]:
def compute_z_score(df):
    df["z"] = (df["close"] - df["vwap"]) / df["rolling_std"]
    return df

### 3.3 Bollinger Bands

In [7]:
def compute_moving_avg(df, n=30):
    df["moving_avg"] = df["close"].rolling(window=n).mean()
    return df

In [8]:
def compute_bb(df, k=2):
    df["bb_upper"] = df["vwap"] + k * df["rolling_std"]
    df["bb_lower"] = df["vwap"] - k * df["rolling_std"]
    return df

### 3.4 RSI

In [9]:
def compute_rsi(df, n_rsi=13):
    df["rsi"] = ta.rsi(df["close"], length=n_rsi)
    return df

### 3.5 Transaction Log
* We initialize a log with a cash amount of choice and a position of 0
* We use a helper function to calculate the real time equity based on position and price

In [10]:
def create_log(df_data, cash):
    df_log = pd.DataFrame(np.nan, index=df_data.index, columns=["cash", "position", "price", "equity", "status"])
    df_log.at[df_log.index[0], "cash"] = cash
    df_log.at[df_log.index[0], "position"] = 0
    df_log.at[df_log.index[0], "price"] = df_data.at[df_data.index[0], "close"]
    df_log["status"] = "OK"
    return df_log

In [11]:
def update_equity(df):
    df["equity"] = df["position"] * df["price"] + df["cash"]
    return df

### 3.6 Paper Broker Orders
* Because of the small size of the trade, each order is "all in" for now
* We simultaneously write to the log whenever an order is executed

In [12]:
def broker_buy(df_data, df_log, quantity, index):
    buy_price = df_data.at[index, "close"]
    df_log.at[index, "price"] = buy_price
    df_log.at[index, "position"] += quantity
    df_log.at[index, "cash"] -= buy_price * quantity
    update_equity(df_log)
    return df_log

In [13]:
def broker_sell(df_data, df_log, quantity, index):
    sell_price = df_data.at[index, "close"]
    df_log.at[index, "price"] = sell_price
    df_log.at[index, "position"] -= quantity
    df_log.at[index, "cash"] += sell_price * quantity
    update_equity(df_log)
    return df_log

### 3.7 Strategy Execution
* For now, the execution only works with the log because of the paper broker orders
* We will set the entry condition according to Bollinger Bands, but equivalently we can use z score as mentioned.

In [14]:
def bbrsi_entry(df_data, df_log, index, rsi=30):
    if df_log.at[index, "status"] == "OK" and df_data.at[index, "close"] <= df_data.at[index, "bb_lower"] and df_data.at[index, "rsi"] <= rsi and df_log.at[index, "cash"] > 0:
        quantity = df_log.at[index, "cash"] // df_data.at[index, "close"]
        df_log = broker_buy(df_data, df_log, quantity, index)
    return df_log

In [15]:
def bbrsi_exit(df_data, df_log, index, rsi=50):
    if df_data.at[index, "close"] >= df_data.at[index, "vwap"] or df_data.at[index, "rsi"] >= rsi:
        quantity = df_log.at[index, "position"]
        df_log = broker_sell(df_data, df_log, quantity, index)
    return df_log

In [16]:
def vwap_sigma_stop(df_data, df_log, index, z_stop=-3.0):
    if df_log.at[index, "position"] != 0 and df_data.at[index, "z"] <= z_stop:
        quantity = df_log.at[index, "position"]
        df_log = broker_sell(df_data, df_log, quantity, index)
        df_log.loc[index:, "status"] = "Cooldown"
    return df_log

In [17]:
# def vwap_time_stop(df_data, df_log, index, stop_interval=120):

In [18]:
def vwap_reset(df_data, df_log, index, z_reset=-0.2):
    if df_log.at[index, "status"] == "Cooldown" and df_data.at[index, "z"] >= z_reset:
        df_log.loc[index:, "status"] = "OK"
    return df_log

In [19]:
def vwap_normalization(df_log, normalization_interval=60):
    start = df_log.index[0]
    end = start + pd.Timedelta(minutes=normalization_interval)
    df_log.loc[start:end, "status"] = "Normalization"
    return df_log

In [20]:
def flatten(df_data, df_log):
    last_index = df_log.index[-1]
    if df_log.at[last_index, "position"] != 0:
        quantity = df_log.at[last_index, "position"]
        df_log = broker_sell(df_data, df_log, quantity, last_index)
    return df_log

## 4 Backtest

In [21]:
backtest_data = datafeed("data/2025-09-03.csv", "SPY")

backtest_data = compute_typical_price(backtest_data)
backtest_data = compute_vwap(backtest_data)
backtest_data = compute_rolling_std(backtest_data)
backtest_data = compute_z_score(backtest_data)

backtest_data = compute_moving_avg(backtest_data)
backtest_data = compute_bb(backtest_data)
backtest_data = compute_rsi(backtest_data)

In [22]:
backtest_log = create_log(backtest_data, 10000.00)

In [23]:
backtest_log = vwap_normalization(backtest_log)

for idx, i in enumerate(backtest_log.index):
    if idx == 0:
        continue
        
    prev_i = backtest_log.index[idx-1]
    backtest_log.at[i, "cash"] = backtest_log.at[prev_i, "cash"]
    backtest_log.at[i, "position"] = backtest_log.at[prev_i, "position"]
    backtest_log.at[i, "price"] = backtest_data.at[i, "close"]
    
    backtest_log = bbrsi_entry(backtest_data, backtest_log, i)
    backtest_log = bbrsi_exit(backtest_data, backtest_log, i)
    backtest_log = vwap_sigma_stop(backtest_data, backtest_log, i)
    backtest_log = vwap_reset(backtest_data, backtest_log, i)

backtest_log = flatten(backtest_data, backtest_log)
backtest_log = update_equity(backtest_log)

## 5 Result Visualization

In [24]:
backtest_data

,ticker,volume,open,close,high,low,transactions,typical,vwap,rolling_std,z,moving_avg,bb_upper,bb_lower,rsi
window_start,,,,,,,,,,,,,,,
2025-09-03 09:30:00,SPY,922,643.12,643.1100,643.12,643.10,19,643.110000,643.110000,NaN,NaN,NaN,NaN,NaN,NaN
2025-09-03 09:32:00,SPY,531,643.04,642.9600,643.04,642.96,13,642.986667,643.064928,NaN,NaN,NaN,NaN,NaN,0.000000
2025-09-03 09:33:00,SPY,2498,642.97,643.0700,643.10,642.97,51,643.046667,643.053382,NaN,NaN,NaN,NaN,NaN,5.759162
2025-09-03 09:34:00,SPY,688,643.07,643.0700,643.07,643.07,14,643.070000,643.055847,NaN,NaN,NaN,NaN,NaN,5.759162
2025-09-03 09:35:00,SPY,616,643.00,642.9900,643.01,642.99,12,642.996667,643.048910,NaN,NaN,NaN,NaN,NaN,5.489326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-03 15:56:00,SPY,59704,642.34,642.4000,642.45,642.32,1149,642.390000,642.883695,0.192396,-2.514066,642.563247,643.268486,642.498904,43.437024
2025-09-03 15:57:00,SPY,76123,642.37,642.1700,642.39,642.12,1296,642.226667,642.881887,0.202626,-3.513306,642.544913,643.287139,642.476635,38.716664
2025-09-03 15:58:00,SPY,61578,642.18,642.2799,642.35,642.18,1243,642.269967,642.880528,0.207506,-2.894507,642.534577,643.295540,642.465516,41.980448


In [25]:
backtest_log

,cash,position,price,equity,status
window_start,,,,,
2025-09-03 09:30:00,10000.0,0.0,643.1100,10000.0,Normalization
2025-09-03 09:32:00,10000.0,0.0,642.9600,10000.0,Normalization
2025-09-03 09:33:00,10000.0,0.0,643.0700,10000.0,Normalization
2025-09-03 09:34:00,10000.0,0.0,643.0700,10000.0,Normalization
2025-09-03 09:35:00,10000.0,0.0,642.9900,10000.0,Normalization
...,...,...,...,...,...
2025-09-03 15:56:00,10002.4,0.0,642.4000,10002.4,OK
2025-09-03 15:57:00,10002.4,0.0,642.1700,10002.4,OK
2025-09-03 15:58:00,10002.4,0.0,642.2799,10002.4,OK


## 6 Notes

### 6.1 To-Do List
* Adjust `datafeed` to accept real time data from `polygon.io`
* Update broker order functions to connect with broker API
* Optimize `enumerate` in backtest
* Implement `vwap_time_stop` function
* Figure out the live minute bar vs closing price in backtest

### 6.2 Caveats
* Many intraday traders only watch VWAP and Bollinger Bands
* If we require both VWAP z-score extreme and RSI extreme, signals become rare
* If we are aiming for higher win-rate, fewer trades, "set it and forget it," then RSI filter may be useful
* Experiment with softer RSI filters (40/60) to balance between signal quality and quantity